In [1]:
import pandas as pd

In [3]:
user_first_withdrawals_query = None

with open('user_first_withdrawal.sql', 'r') as f:
    user_first_withdrawals_query = f.read()

user_first_withdrawals_df = pd.read_gbq(query=user_first_withdrawals_query)

In [11]:
user_first_withdrawals_df.to_json('gs://friktion-users-prod/user-first-withdrawals.json', orient='records', date_format='iso', lines=True, date_unit='s')

In [5]:
user_first_deposits_query = None

with open('user_first_deposit.sql', 'r') as f:
    user_first_deposits_query = f.read()

user_first_deposits_df = pd.read_gbq(query=user_first_deposits_query)

In [6]:
user_first_deposits_df

,user_address,first_deposit_date,first_deposit_epoch,first_deposit_amount,first_deposit_token
0,4jfevPD5Q6NYCtBXCxcpxLWw4J2eydt71BB5uPG4eZJb,2022-03-27,14,15398.243360,"[AVAX, AVAX, BTC, ETH, FTT, SOL, mSOL]"
1,E5JYULNnbYvvwGGjiMzZDVeM6bbLNpm1ATSuJLwAgZLq,2022-03-27,14,516.580000,[AVAX]
2,2Mn4ZSw3JL6oS3U2zTJsQYYJFD1T2efeWaup5UopYMjV,2022-03-28,14,93.038222,"[AVAX, AVAX]"
3,2EjEpLvt5rbYVZSyxHTbiUZDpuR5JiA1RFSGLVvWj6oJ,2022-03-28,14,2021.969276,"[AVAX, AVAX]"
4,4iLRfA59UtAap3FBzPzL1CJWAWkUkJbrdug3GMMC4XmJ,2022-03-28,14,788.316984,"[AVAX, LUNA, USDC, USDC, USDC]"
...,...,...,...,...,...
13049,D1Ns1rnqTumriCP9sDLPgpa3HhxNh5cJgZ4vMJRkGLk4,2022-04-03,15,185.689564,[tsUSDC]
13050,3mdXjFceuhtMzS6wAjW21y174VSRuJprf9BLZUNqVLrj,2022-04-06,15,994.008189,[tsUSDC]
13051,5bpjaUQndFaM3EiQLkcdQ3HBTm7EpE7kDoBjh2pbSUUY,2022-04-07,15,9930.865558,[tsUSDC]
13052,5g7YUjSjLnTo6QNZKpifgy4CdYNsP9Yh28WS2vxpk6UU,2022-04-07,15,122.546598,[tsUSDC]


In [12]:
user_first_deposits_df.to_json('gs://friktion-users-prod/user-first-deposits.json', orient='records', date_format='iso', lines=True)

In [13]:
!bq load --source_format=NEWLINE_DELIMITED_JSON --autodetect users.fact_user_first_withdrawal "gs://friktion-users-prod/user-first-withdrawals.json"
!bq load --source_format=NEWLINE_DELIMITED_JSON --autodetect users.fact_user_first_deposit "gs://friktion-users-prod/user-first-deposits.json"

Waiting on bqjob_r1111c2f8cb98b553_0000018049b09b02_1 ... (1s) Current status: DONE   
Waiting on bqjob_r67bba061c9493c90_0000018049b0aa89_1 ... (2s) Current status: DONE   


In [10]:
from google.cloud import bigquery

# Construct a BigQuery client object.
client = bigquery.Client()

# DONE(developer): Set dataset_id to the ID of the dataset to create.
dataset_id = "{}.users".format(client.project)

# Construct a full Dataset object to send to the API.
dataset = bigquery.Dataset(dataset_id)

# DONE(developer): Specify the geographic location where the dataset should reside.
dataset.location = "US"

# Send the dataset to the API for creation, with an explicit timeout.
# Raises google.api_core.exceptions.Conflict if the Dataset already
# exists within the project.
dataset = client.create_dataset(dataset, timeout=30)  # Make an API request.
print("Created dataset {}.{}".format(client.project, dataset.dataset_id))

Created dataset lyrical-amulet-337502.users


In [14]:
user_firsts_query = None

with open('user_firsts_table.sql', 'r') as f:
    user_firsts_query = f.read()

user_firsts_df = pd.read_gbq(query=user_firsts_query)

user_firsts_df.to_json('gs://friktion-users-prod/user-firsts.json', orient='records', date_format='iso', lines=True, date_unit='s')

In [15]:
!bq load --source_format=NEWLINE_DELIMITED_JSON --autodetect users.fact_user_firsts "gs://friktion-users-prod/user-firsts.json"

Waiting on bqjob_re9cd2bf98609d47_0000018049b178bf_1 ... (2s) Current status: DONE   


In [35]:
max_date_df = pd.read_gbq(query='(select max(date(deposit_initiated_ts)) from `transactions.fact_deposits`)')
max_date = max_date_df.iloc[0,0]

_user_table_script_query = None

with open('user_table_sript.sql', 'r') as f:
    _user_table_script_query = f.read()
    
def collect_and_write_user_table(user_address, first_deposit_date, max_date):
    from google.cloud import bigquery

    # Construct a BigQuery client object.
    client = bigquery.Client()

    # Run a SQL script.
    sql_script = _user_table_script_query
    
    job_config = bigquery.QueryJobConfig(
        query_parameters=[
            bigquery.ScalarQueryParameter("user_address", "STRING", user_address),
            bigquery.ScalarQueryParameter("first_deposit_dt", "DATE", first_deposit_date),
            bigquery.ScalarQueryParameter("max_date", "DATE", max_date)
        ]
    )

    parent_job = client.query(sql_script, job_config=job_config)

    # Wait for the whole script to finish.
    rows_iterable = parent_job.result()
    # print("Script created {} child jobs.".format(parent_job.num_child_jobs))

    # Fetch result rows for the final sub-job in the script.
    rows = list(rows_iterable)
    
    print_true = (len(rows) > abs((user_first_deposits_df['first_deposit_date'].min() - max_date).days)+1)
    
    if print_true:
        print(
            f'{len(rows)} rows of user data were collected for user {user_address}'
            )


    # Fetch jobs created by the SQL script.
    child_jobs_iterable = client.list_jobs(parent_job=parent_job)
    for child_job in child_jobs_iterable:
        child_rows = list(child_job.result())
        # print(
        #     "Child job with ID {} produced {} row(s).".format(
        #         child_job.job_id, len(child_rows)
        #     )
        # )
        
    user_df = None
    for job in client.list_jobs(parent_job=parent_job):
        user_df = client.get_job(job.job_id).to_dataframe().sort_values('as_of_date')
        break
    
    user_df.to_json(f'gcs://friktion-users-prod/user-{user_address}.json', orient='records', date_format='iso', lines=True, date_unit='s')
    if print_true:
        print(f'Written to gcs://friktion-users-prod/user-{user_address}.json')

In [40]:
user_first_deposits_df.iloc[2001:3000,:].to_json(f'user_first_deposits_part_3.json', orient='records', date_format='iso', lines=True, date_unit='s')
user_first_deposits_df.iloc[3001:4000,:].to_json(f'user_first_deposits_part_4.json', orient='records', date_format='iso', lines=True, date_unit='s')
user_first_deposits_df.iloc[4001:5000,:].to_json(f'user_first_deposits_part_5.json', orient='records', date_format='iso', lines=True, date_unit='s')
user_first_deposits_df.iloc[5001:6000,:].to_json(f'user_first_deposits_part_6.json', orient='records', date_format='iso', lines=True, date_unit='s')
user_first_deposits_df.iloc[6001:7000,:].to_json(f'user_first_deposits_part_7.json', orient='records', date_format='iso', lines=True, date_unit='s')
user_first_deposits_df.iloc[7001:8000,:].to_json(f'user_first_deposits_part_8.json', orient='records', date_format='iso', lines=True, date_unit='s')
user_first_deposits_df.iloc[8001:9000,:].to_json(f'user_first_deposits_part_9.json', orient='records', date_format='iso', lines=True, date_unit='s')
user_first_deposits_df.iloc[9001:10000,:].to_json(f'user_first_deposits_part_10.json', orient='records', date_format='iso', lines=True, date_unit='s')
user_first_deposits_df.iloc[10001:11000,:].to_json(f'user_first_deposits_part_11.json', orient='records', date_format='iso', lines=True, date_unit='s')
user_first_deposits_df.iloc[11001:12000,:].to_json(f'user_first_deposits_part_12.json', orient='records', date_format='iso', lines=True, date_unit='s')
user_first_deposits_df.iloc[12001:13000,:].to_json(f'user_first_deposits_part_13.json', orient='records', date_format='iso', lines=True, date_unit='s')
user_first_deposits_df.iloc[13001:,:].to_json(f'user_first_deposits_part_14.json', orient='records', date_format='iso', lines=True, date_unit='s')


In [36]:
from tqdm.notebook import tqdm, tnrange
for row in tqdm(user_first_deposits_df.itertuples()):
    collect_and_write_user_table(row.user_address, row.first_deposit_date, max_date)

0it [00:00, ?it/s]

224 rows of user data were collected for user 36dDm4AaPz4ZNLf732DXKRr8yUKyrQMBd1oFsZvBKZJy
Written to gcs://friktion-users-prod/user-36dDm4AaPz4ZNLf732DXKRr8yUKyrQMBd1oFsZvBKZJy.json


KeyboardInterrupt: 

In [11]:

from google.cloud import bigquery

# Construct a BigQuery client object.
client = bigquery.Client()

# Run a SQL script.
sql_script = _user_table_script_query
user_address = user_first_deposits_df.iloc[0,:].user_address
first_deposit_date = user_first_deposits_df.iloc[0,:].first_deposit_dt.isoformat()

job_config = bigquery.QueryJobConfig(
    query_parameters=[
        bigquery.ScalarQueryParameter("user_address", "STRING", user_address),
        bigquery.ScalarQueryParameter("first_deposit_dt", "DATE", first_deposit_date),
    ]
)

parent_job = client.query(sql_script, job_config=job_config)

# Wait for the whole script to finish.
rows_iterable = parent_job.result()
print("Script created {} child jobs.".format(parent_job.num_child_jobs))

# Fetch result rows for the final sub-job in the script.
rows = list(rows_iterable)
print(
    f'{len(rows)} rows of user data were collected for user {user_address}'
    )


# Fetch jobs created by the SQL script.
child_jobs_iterable = client.list_jobs(parent_job=parent_job)
for child_job in child_jobs_iterable:
    child_rows = list(child_job.result())
    print(
        "Child job with ID {} produced {} row(s).".format(
            child_job.job_id, len(child_rows)
        )
    )
    

Script created 10 child jobs.
94 rows of user data were collected for user 287TuzSrFGtrQbFPuXkwuQwX2nmwSpHxAMdmtKwfbzBv
Child job with ID script_job_fb346060af2d4aeda08cc53928b982c7_9 produced 94 row(s).
Child job with ID script_job_ca2b50f487a38630880d60e05d905484_8 produced 0 row(s).
Child job with ID script_job_9592d23046f67673e4d4e85c1b91aa8f_7 produced 0 row(s).
Child job with ID script_job_008f509cc4d1ff6186aff40a2b97ea74_6 produced 0 row(s).
Child job with ID script_job_468ee2653041f7adfc4f231d7619cf60_5 produced 1 row(s).
Child job with ID script_job_389b9eaf0156bf396492a2fd7d555566_4 produced 1 row(s).
Child job with ID script_job_9c24238ac0ffc78fbe006d7a603527fa_3 produced 1 row(s).
Child job with ID script_job_98ff4e72096afaa0f525e54927b28053_2 produced 0 row(s).
Child job with ID script_job_f1b3d40b0f412668bd9ab9ca3e3f1efc_1 produced 1 row(s).
Child job with ID script_job_5019d73e7d498517fd4b2b4593216e85_0 produced 1 row(s).


In [18]:
user_df = None
for job in client.list_jobs(parent_job=parent_job):
    user_df = client.get_job(job.job_id).to_dataframe().sort_values('as_of_date')
    break
    
user_df

,user_address,as_of_date,total_deposits_usd,total_withdrawals_usd,total_value_locked_USD,first_deposit_date,first_deposit_amount,first_deposit_token,first_deposit_epoch,first_withdrawal_date,...,last_deposit_amt,last_deposit_epoch,last_deposit_token,last_deposit_date,days_since_last_deposit,tvl_delta_30_day,tvl_delta_60_day,tvl_delta_90_day,has_churned,churn_date
0,287TuzSrFGtrQbFPuXkwuQwX2nmwSpHxAMdmtKwfbzBv,2021-12-29,0.000363,0.0,0.000363,2021-12-29,0.000363,[BTC],1,None,...,0.000363,1,BTC,2021-12-29,0,0.000363,0.000363,0.000363,False,None
1,287TuzSrFGtrQbFPuXkwuQwX2nmwSpHxAMdmtKwfbzBv,2021-12-30,0.000363,0.0,0.000363,2021-12-29,0.000363,[BTC],1,None,...,0.000363,1,BTC,2021-12-29,1,0.000363,0.000363,0.000363,False,None
2,287TuzSrFGtrQbFPuXkwuQwX2nmwSpHxAMdmtKwfbzBv,2021-12-31,0.000363,0.0,0.000363,2021-12-29,0.000363,[BTC],1,None,...,0.000363,1,BTC,2021-12-29,2,0.000363,0.000363,0.000363,False,None
3,287TuzSrFGtrQbFPuXkwuQwX2nmwSpHxAMdmtKwfbzBv,2022-01-01,0.000363,0.0,0.000363,2021-12-29,0.000363,[BTC],1,None,...,0.000363,1,BTC,2021-12-29,3,0.000363,0.000363,0.000363,False,None
4,287TuzSrFGtrQbFPuXkwuQwX2nmwSpHxAMdmtKwfbzBv,2022-01-02,0.000363,0.0,0.000363,2021-12-29,0.000363,[BTC],1,None,...,0.000363,1,BTC,2021-12-29,4,0.000363,0.000363,0.000363,False,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,287TuzSrFGtrQbFPuXkwuQwX2nmwSpHxAMdmtKwfbzBv,2022-03-28,0.000363,0.0,0.000363,2021-12-29,0.000363,[BTC],1,None,...,0.000363,1,BTC,2021-12-29,89,NaN,NaN,0.000363,False,None
90,287TuzSrFGtrQbFPuXkwuQwX2nmwSpHxAMdmtKwfbzBv,2022-03-29,0.000363,0.0,0.000363,2021-12-29,0.000363,[BTC],1,None,...,0.000363,1,BTC,2021-12-29,90,NaN,NaN,0.000363,False,None
91,287TuzSrFGtrQbFPuXkwuQwX2nmwSpHxAMdmtKwfbzBv,2022-03-30,0.000363,0.0,0.000363,2021-12-29,0.000363,[BTC],1,None,...,0.000363,1,BTC,2021-12-29,91,NaN,NaN,NaN,False,None
92,287TuzSrFGtrQbFPuXkwuQwX2nmwSpHxAMdmtKwfbzBv,2022-03-31,0.000363,0.0,0.000363,2021-12-29,0.000363,[BTC],1,None,...,0.000363,1,BTC,2021-12-29,92,NaN,NaN,NaN,False,None


In [19]:
user_df.to_parquet(f'gcs://friktion-users/user-{user_address}.parquet', index=False)                   

In [20]:
loaded_df = pd.read_parquet(f'')
loaded_df

,user_address,as_of_date,total_deposits_usd,total_withdrawals_usd,total_value_locked_USD,first_deposit_date,first_deposit_amount,first_deposit_token,first_deposit_epoch,first_withdrawal_date,...,last_deposit_amt,last_deposit_epoch,last_deposit_token,last_deposit_date,days_since_last_deposit,tvl_delta_30_day,tvl_delta_60_day,tvl_delta_90_day,has_churned,churn_date
0,287TuzSrFGtrQbFPuXkwuQwX2nmwSpHxAMdmtKwfbzBv,2021-12-29,0.000363,0.0,0.000363,2021-12-29,0.000363,[BTC],1,None,...,0.000363,1,BTC,2021-12-29,0,0.000363,0.000363,0.000363,False,None
1,287TuzSrFGtrQbFPuXkwuQwX2nmwSpHxAMdmtKwfbzBv,2021-12-30,0.000363,0.0,0.000363,2021-12-29,0.000363,[BTC],1,None,...,0.000363,1,BTC,2021-12-29,1,0.000363,0.000363,0.000363,False,None
2,287TuzSrFGtrQbFPuXkwuQwX2nmwSpHxAMdmtKwfbzBv,2021-12-31,0.000363,0.0,0.000363,2021-12-29,0.000363,[BTC],1,None,...,0.000363,1,BTC,2021-12-29,2,0.000363,0.000363,0.000363,False,None
3,287TuzSrFGtrQbFPuXkwuQwX2nmwSpHxAMdmtKwfbzBv,2022-01-01,0.000363,0.0,0.000363,2021-12-29,0.000363,[BTC],1,None,...,0.000363,1,BTC,2021-12-29,3,0.000363,0.000363,0.000363,False,None
4,287TuzSrFGtrQbFPuXkwuQwX2nmwSpHxAMdmtKwfbzBv,2022-01-02,0.000363,0.0,0.000363,2021-12-29,0.000363,[BTC],1,None,...,0.000363,1,BTC,2021-12-29,4,0.000363,0.000363,0.000363,False,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,287TuzSrFGtrQbFPuXkwuQwX2nmwSpHxAMdmtKwfbzBv,2022-03-28,0.000363,0.0,0.000363,2021-12-29,0.000363,[BTC],1,None,...,0.000363,1,BTC,2021-12-29,89,NaN,NaN,0.000363,False,None
90,287TuzSrFGtrQbFPuXkwuQwX2nmwSpHxAMdmtKwfbzBv,2022-03-29,0.000363,0.0,0.000363,2021-12-29,0.000363,[BTC],1,None,...,0.000363,1,BTC,2021-12-29,90,NaN,NaN,0.000363,False,None
91,287TuzSrFGtrQbFPuXkwuQwX2nmwSpHxAMdmtKwfbzBv,2022-03-30,0.000363,0.0,0.000363,2021-12-29,0.000363,[BTC],1,None,...,0.000363,1,BTC,2021-12-29,91,NaN,NaN,NaN,False,None
92,287TuzSrFGtrQbFPuXkwuQwX2nmwSpHxAMdmtKwfbzBv,2022-03-31,0.000363,0.0,0.000363,2021-12-29,0.000363,[BTC],1,None,...,0.000363,1,BTC,2021-12-29,92,NaN,NaN,NaN,False,None
